In [ ]:
import numpy as np
import os
from numpy.fft import fft, ifft, fftshift

def extract_dimensions(file_name):
    parts = file_name.split('_')
    dimensions = []
    for part in parts:
        if 'z=' in part or 'x=' in part or 'y=' in part:
            number = int(part.split('=')[-1])
            dimensions.append(number)
    return tuple(dimensions)

def read_tomogram(file_path, dimensions):
    depth, height, width = dimensions
    with open(file_path, 'rb') as file:
        tomogram = np.fromfile(file, dtype='single')
        tomogram = tomogram.reshape((depth, height, width),order='F')
    return tomogram

In [ ]:
pathcomplex = r'C:\Users\USER\Documents\GitHub\[s.fovea]11bscan'
artifact_files = os.listdir(pathcomplex)
for imag_file, real_file in zip(artifact_files[::2], artifact_files[1::2]):
        real_file_path = os.path.join(pathcomplex, real_file)
        imag_file_path = os.path.join(pathcomplex, imag_file)
        dimensions = extract_dimensions(real_file[:-4])
        tomReal = read_tomogram(real_file_path, dimensions)
        tomImag = read_tomogram(imag_file_path, dimensions)
        tomcc = tomReal + 1j * tomImag
        fringescc = fftshift(ifft(tomcc,axis=0),axes=0)
        fringescc = np.stack((fringescc.real,fringescc.imag),axis=3)
        del tomImag, tomReal
# fringescc = fftshift(ifft(fftshift(tom,axes=0),axis=0),axes=0)
# fringescc = fftshift(ifft(tomcc,axis=0),axes=0)

pathcomplex = r'C:\Users\USER\Documents\GitHub\[s.fovea]11bscanNoartifacts'
artifact_files = os.listdir(pathcomplex)
for imag_file, real_file in zip(artifact_files[::2], artifact_files[1::2]):
        real_file_path = os.path.join(pathcomplex, real_file)
        imag_file_path = os.path.join(pathcomplex, imag_file)
        dimensions = extract_dimensions(real_file[:-4])
        tomReal = read_tomogram(real_file_path, dimensions)
        tomImag = read_tomogram(imag_file_path, dimensions)
        tom = tomReal + 1j * tomImag
        fringes = fftshift(ifft(tom,axis=0),axes=0)
        fringes = np.stack((fringes.real,fringes.imag),axis=3)
        del tomImag, tomReal
# fringescc = fftshift(ifft(fftshift(tom,axes=0),axis=0),axes=0)
# fringes = fftshift(ifft(tom,axis=0),axes=0)

fringesccHilbert = hilbert(fringescc[:,:,:,0],axis=0)
fringesccAnalytic = fringescc[:,:,:,0] + 1j*fringesccHilbert
fringesccAnalytic = np.stack((fringesccAnalytic.real,fringesccAnalytic.imag),axis=3)
del fringesccHilbert